# Maximum Ratio Combining

_Author:_ Karl-Ludwig Besser, Technische Universität Braunschweig

This notebook is part of the paper "On Fading Channel Dependency Structures with a Positive Zero-Outage Capacity" ([arXiv:XXX](https://arxiv.org)).  
If you use any of this work, please cite the above paper.

> If you are not familiar with Jupyter notebooks: The easiest way to use this notebook interactively, is to hit `Kernel --> Restart & Run All` in the menu. The will execute all cells and enable the interactive elements of the plots.  
> Alternatively, you can execute the cells one by one using Shift+Return

In [ ]:
import numpy as np
from scipy import stats
from scipy import optimize
%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact

In [ ]:
from maximum_ratio_combining import (max_zoc_inner_bound_mrc_homog, max_zoc_outer_bound_mrc_homog,
                                     zoc_copula_t_mrc_heterog, max_zoc_outer_bound_joint_mix_mrc_homog)
from utils import w_copula

In [ ]:
@np.vectorize
def my_copula2(a, b, t=.5):
    if a < t and b < t:
        c = np.maximum(a+b-t, 0)
    else:
        c = np.minimum(a, b)
    return c

# General Idea

The following plot illustrates the general idea of finding the ZOC for dependent fading channels.

In the example, the joint distribution of two countermonotonic Rayleigh fading channels is shown. The first link ($x$-axis) has an SNR of $5$ dB while the second link ($y$-axis) has an SNR of $-5$ dB.

You can see the following in the plots:
- The red line highlights the boundary of the support.
- The white dashed line corresponds to $s_{\text{MRC}}=2^R-1$ for MRC at the receiver. For MRC, it is given by $s_{\text{MRC}}=x+y$. The details can be found in the paper. 
- The blue dashed line corresponds to $s_{\text{SC}}=2^R-1$ for SC at the receiver. For SC, it is given by $s_{\text{SC}}=\max\{x, y\}$. The details can be found in the paper.

The maximum $s$ that is still below the red line, is the corresponding to the ZOC.

In [ ]:
def plot_two_d_problem():
    a, stepsize = np.linspace(0, 2, 150, retstep=True)
    A, B = np.meshgrid(a, a)
    fig, ax = plt.subplots(1,1)
    snr_x_db=5
    snr_y_db=-5
    
    def update_plot(s_mrc=0, s_sc=0):
        snr_x = 10**(snr_x_db/10.)
        snr_y = 10**(snr_y_db/10.)
        rv_x = stats.expon(scale=snr_x)
        rv_y = stats.expon(scale=snr_y)
        cdf_x = rv_x.cdf(A)
        cdf_y = rv_y.cdf(B)
        joint_cdf = w_copula(cdf_x, cdf_y)
        ax.clear()
        _gradx = np.gradient(joint_cdf, stepsize, axis=0)
        joint_pdf = np.gradient(_gradx, stepsize, axis=1)
        ax.set_xlim([min(a), max(a)])
        ax.set_ylim([min(a), max(a)])
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.pcolormesh(A, B, joint_pdf, vmin=0, shading="auto")
        ax.plot(a, rv_y.ppf(1-rv_x.cdf(a)), 'r-')
        ax.plot([0, s_sc, s_sc], [s_sc, s_sc, 0], 'c--')
        ax.plot([0, s_mrc], [s_mrc, 0], 'w--')
        
    interact(update_plot, s_mrc=(0, 2, 0.01), s_sc=(0, 1, .01))

In [ ]:
plot_two_d_problem()

# Rayleigh Fading

The first example is Rayleigh fading. In this case, the channel gains $|H_i|^2=X_i$ are exponentially distributed according to $X_i\sim\exp(1/\rho_i)$, where $\rho_i$ denotes the SNR of transmission link $i$.

In [ ]:
from rayleigh_fading import zoc_copula_t_mrc_heterog_rayleigh

## Two-Dimensional Heterogeneous Case

In [ ]:
def plot_heterog_rayleigh():
    a, stepsize = np.linspace(0, 3, 150, retstep=True)
    A, B = np.meshgrid(a, a)
    fig, ax = plt.subplots(1,1)
    
    def update_plot(t=.5, snr_x_db=0, snr_y_db=0):
        snr_x = 10**(snr_x_db/10.)
        snr_y = 10**(snr_y_db/10.)
        rv_x = stats.expon(scale=snr_x)
        rv_y = stats.expon(scale=snr_y)
        cdf_x = rv_x.cdf(A)
        cdf_y = rv_y.cdf(B)
        joint_cdf = my_copula2(cdf_x, cdf_y, t=t)
        ax.clear()
        _gradx = np.gradient(joint_cdf, stepsize, axis=0)
        joint_pdf = np.gradient(_gradx, stepsize, axis=1)
        ax.set_xlim([min(a), max(a)])
        ax.set_ylim([min(a), max(a)])
        ax.set_xlabel("$X_1$")
        ax.set_ylabel("$X_2$")
        ax.pcolormesh(A, B, joint_pdf, vmin=0, shading="auto")
        ax.plot(a, rv_y.ppf(t-rv_x.cdf(a)), 'r-')
        zoc_mrc = zoc_copula_t_mrc_heterog_rayleigh(t, 1/snr_x, 1/snr_y)
        s_zoc = 2**zoc_mrc - 1
        ax.plot([0, s_zoc], [s_zoc, 0], 'w--')
        
    interact(update_plot, t=(0, 1, 0.1), snr_x_db=(-5, 10, 1), snr_y_db=(-5, 10, 1))

In the two-dimensional case, a positive ZOC is possible, if the two channel gains $(X_1, X_2)$ follow a specific copula, e.g., $C_t$.

In the plot below, this joint distribution is shown. You can change the parameter $t$ and the SNRs $\rho_i$.
A value of $t=0$ corresponds to comonotonic channel gains; while a value of $t=1$ corresponds to countermonotonic channel gains.

In [ ]:
plot_heterog_rayleigh()

In [ ]:
def plot_zero_out_rayleigh_grid():
    snr_db = np.linspace(-5, 15, 50)
    SNR_X_DB, SNR_Y_DB = np.meshgrid(snr_db, snr_db)
    SNR_X = 10**(SNR_X_DB/10.)
    SNR_Y = 10**(SNR_Y_DB/10.)
    fig, ax = plt.subplots(1,1)
    def plot_copula(t=.5):
        LX = 1/(SNR_X)
        LY = 1/(SNR_Y)
        ax.clear()
        ax.set_xlim([min(snr_db), max(snr_db)])
        ax.set_ylim([min(snr_db), max(snr_db)])
        ax.set_xlabel("$\\rho_1$ [dB]")
        ax.set_ylabel("$\\rho_2$ [dB]")
        capac = zoc_copula_t_mrc_heterog_rayleigh(t, LX, LY)
        ax.pcolormesh(SNR_X_DB, SNR_Y_DB, capac, vmin=0, alpha=.5, shading='auto')
        _contour = ax.contour(SNR_X_DB, SNR_Y_DB, capac, vmin=0)
        ax.clabel(_contour, inline=1, fontsize=9)
    interact(plot_copula, t=(0, 1, .01))

In [ ]:
plot_zero_out_rayleigh_grid()

## N-Dimensional Homogeneous Case

In the $n$-dimensional case, we are able to provide bounds on the maximum ZOC for homogenoues links, i.e, all $X_i$ follow the same marginal distribution.

Detailed information about the bounds can be found in the paper.

In [ ]:
def plot_homog_rayleigh():
    n = np.arange(2, 21)
    fig, ax = plt.subplots(1,1)
    funcs = {"Inner Bound": max_zoc_inner_bound_mrc_homog,
             "Outer Bound - Joint Mixability": max_zoc_outer_bound_joint_mix_mrc_homog,
             "Outer Bound - Frechet-Hoeffding": max_zoc_outer_bound_mrc_homog}
    plots = {k: ax.plot(n, np.zeros_like(n), 'o-', label=k)[0] for k in funcs.keys()}
    ax.legend()
    ax.set_ylim([0, 10])
    ax.set_xlabel("n")
    ax.set_ylabel("Max ZOC")
    
    def update_plot(snr_db=5.):
        snr = 10**(snr_db/10.)
        dist = stats.expon(scale=snr)
        for _name, _func in funcs.items():
            plots[_name].set_ydata(_func(dist, n))
        
    interact(update_plot, snr_db=(0., 10., .2))

In the plot below, the different bounds on the maximum ZOC for $n$ homogeneous Rayleigh fading links are shown.

In [ ]:
plot_homog_rayleigh()

# Nakagami-m Fading

Similarly to the above [Rayleigh fading example](#Rayleigh-Fading), we provide interactive plots for Nakagami-$m$ fading below.

## Two-Dimensional Heterogeneous Case

In [ ]:
def plot_heterog_nakagami():
    a, stepsize = np.linspace(0, 5, 150, retstep=True)
    A, B = np.meshgrid(a, a)
    fig, ax = plt.subplots(1,1)
    
    def update_plot(t=0.5, snr_x_db=0, snr_y_db=0, m=5):
        snr_x = 10**(snr_x_db/10.)
        snr_y = 10**(snr_y_db/10.)
        rv_x = stats.gamma(a=m, scale=snr_x/m)
        rv_y = stats.gamma(a=m, scale=snr_y/m)
        cdf_x = rv_x.cdf(A)
        cdf_y = rv_y.cdf(B)
        joint_cdf = my_copula2(cdf_x, cdf_y, t=t)
        ax.clear()
        _gradx = np.gradient(joint_cdf, stepsize, axis=0)
        joint_pdf = np.gradient(_gradx, stepsize, axis=1)
        ax.set_xlim([0, 5])
        ax.set_ylim([0, 5])
        ax.pcolormesh(A, B, joint_pdf, vmin=0, shading="auto")
        ax.plot(a, rv_y.ppf(t-rv_x.cdf(a)), 'r-')
        zoc_mrc = zoc_copula_t_mrc_heterog(t, rv_x, rv_y)
        s_zoc = 2**zoc_mrc - 1
        ax.plot([0, s_zoc], [s_zoc, 0], 'w--')
        
    interact(update_plot, t=(0, 1, .1), snr_x_db=(-5, 10, 1), snr_y_db=(-5, 10, 1), m=(1, 10, 1))

In [ ]:
plot_heterog_nakagami()

## N-Dimensional Homogeneous Case

In [ ]:
def plot_homog_nakagami():
    n = np.arange(2, 21)
    fig, ax = plt.subplots(1,1)
    funcs = {"Inner Bound": max_zoc_inner_bound_mrc_homog,
             "Outer Bound - Joint Mixability": max_zoc_outer_bound_joint_mix_mrc_homog,
             "Outer Bound - Frechet-Hoeffding": max_zoc_outer_bound_mrc_homog}
    plots = {k: ax.plot(n, np.zeros_like(n), 'o-', label=k)[0] for k in funcs.keys()}
    ax.legend()
    ax.set_ylim([0, 10])
    ax.set_xlabel("n")
    ax.set_ylabel("Max ZOC")
    
    def update_plot(snr_db=5., m=3):
        snr = 10**(snr_db/10.)
        dist = stats.gamma(a=m, scale=snr/m)
        for _name, _func in funcs.items():
            plots[_name].set_ydata(_func(dist, n))
        
    interact(update_plot, snr_db=(0., 10., .2), m=(1, 10, 1))

In [ ]:
plot_homog_nakagami()